pca_bert_250features

In [2]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [3]:
import warnings 
warnings.filterwarnings("ignore") 

In [4]:
import os
import numpy as np 
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [5]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Bert/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [6]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [7]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [8]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [9]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [10]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [11]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [12]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Bert/bert_withstopwordchangelevels_250_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,240,241,242,243,244,245,246,247,248,249
0,0,-15.959655,7.610355,-4.614255,3.683309,2.789502,-1.250055,-1.805187,-10.514873,-4.086176,...,-0.102584,0.267581,-0.403299,-0.217952,0.057569,0.077332,-0.126021,0.117609,0.029336,0.202135
1,1,-13.495674,4.785825,-0.513378,4.535705,-0.666486,-2.018108,-2.203631,5.270856,-0.704172,...,0.291120,0.006789,-0.202486,0.040521,-0.024232,0.438392,0.089987,-0.014633,-0.013465,-0.049463
2,2,5.556126,-6.071359,15.245860,-6.528944,1.345797,5.089355,-4.129077,-5.236971,-3.814316,...,0.450616,0.431159,-0.480233,-0.170135,0.499157,-0.415463,0.234450,-0.127152,-0.518867,0.272914
3,3,-4.445179,1.293825,-4.332579,-6.459947,3.055774,8.987284,7.318152,-5.308832,-3.731958,...,-0.349662,-0.152237,-0.586068,-0.179928,-0.526360,0.220094,0.222429,-0.038581,0.004379,0.333555
4,4,-12.149864,1.734085,-7.929423,-3.487395,3.067365,4.222421,-4.707823,2.094515,0.077916,...,0.074331,-0.338541,0.019009,-0.229614,-0.123812,0.033707,-0.082511,-0.221210,0.141282,-0.024821


In [13]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,-15.959655,7.610355,-4.614255,3.683309,2.789502,-1.250055,-1.805187,-10.514873,-4.086176,0.203140,...,-0.102584,0.267581,-0.403299,-0.217952,0.057569,0.077332,-0.126021,0.117609,0.029336,0.202135
1,-13.495674,4.785825,-0.513378,4.535705,-0.666486,-2.018108,-2.203631,5.270856,-0.704172,1.341339,...,0.291120,0.006789,-0.202486,0.040521,-0.024232,0.438392,0.089987,-0.014633,-0.013465,-0.049463
2,5.556126,-6.071359,15.245860,-6.528944,1.345797,5.089355,-4.129077,-5.236971,-3.814316,-2.739572,...,0.450616,0.431159,-0.480233,-0.170135,0.499157,-0.415463,0.234450,-0.127152,-0.518867,0.272914
3,-4.445179,1.293825,-4.332579,-6.459947,3.055774,8.987284,7.318152,-5.308832,-3.731958,-2.567814,...,-0.349662,-0.152237,-0.586068,-0.179928,-0.526360,0.220094,0.222429,-0.038581,0.004379,0.333555
4,-12.149864,1.734085,-7.929423,-3.487395,3.067365,4.222421,-4.707823,2.094515,0.077916,1.247958,...,0.074331,-0.338541,0.019009,-0.229614,-0.123812,0.033707,-0.082511,-0.221210,0.141282,-0.024821


In [14]:
y=df['Level '] 

**Train Test Split**

In [15]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 250), (524, 250))

In [16]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.3733121264578591


**Decision Tree**

In [17]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [18]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [19]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [20]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [21]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [22]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.515574,0.515574,0.568807,0.511937,0.349237,0.349237,0.36526,0.341332


**Random Forest**

In [23]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [24]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [25]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [26]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    1.0000    1.0000       361
           2     1.0000    1.0000    1.0000       553
           3     1.0000    1.0000    1.0000       306

    accuracy                         1.0000      1220
   macro avg     1.0000    1.0000    1.0000      1220
weighted avg     1.0000    1.0000    1.0000      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2000    0.0637    0.0966       157
           2     0.4590    0.8734    0.6017       237
           3     0.2609    0.0462    0.0784       130

    accuracy                         0.4256       524
   macro avg     0.3066    0.3278    0.2589       524
weighted avg     0.3322    0.4256    0.3206       524



In [27]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.515574,0.515574,0.568807,0.511937,0.349237,0.349237,0.365260,0.341332
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.425573,0.425573,0.332235,0.320570


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.6086    0.5125    0.5564       361
           2     0.5875    0.7288    0.6505       553
           3     0.6087    0.4575    0.5224       306

    accuracy                         0.5967      1220
   macro avg     0.6016    0.5662    0.5764      1220
weighted avg     0.5990    0.5967    0.5905      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2574    0.1656    0.2016       157
           2     0.4472    0.6076    0.5152       237
           3     0.1980    0.1538    0.1732       130

    accuracy                         0.3626       524
   macro avg     0.3009    0.3090    0.2966       524
weighted avg     0.3285    0.3626    0.3364       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.526230,0.526230,0.651813,0.442314,0.427481,0.427481,0.338740,0.321962
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.429389,0.429389,0.328686,0.329094
2,Adaboost,0.596721,0.596721,0.599029,0.590531,0.362595,0.362595,0.328523,0.336370


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    0.9778    0.9888       361
           2     0.9668    1.0000    0.9831       553
           3     1.0000    0.9641    0.9817       306

    accuracy                         0.9844      1220
   macro avg     0.9889    0.9806    0.9845      1220
weighted avg     0.9849    0.9844    0.9844      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3304    0.2420    0.2794       157
           2     0.4480    0.6540    0.5317       237
           3     0.1905    0.0923    0.1244       130

    accuracy                         0.3912       524
   macro avg     0.3230    0.3295    0.3118       524
weighted avg     0.3489    0.3912    0.3551       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.526230,0.526230,0.651813,0.442314,0.427481,0.427481,0.338740,0.321962
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.429389,0.429389,0.328686,0.329094
2,Adaboost,0.596721,0.596721,0.599029,0.590531,0.362595,0.362595,0.328523,0.336370
3,GBM,0.984426,0.984426,0.984944,0.984438,0.391221,0.391221,0.348875,0.355065


**SVM Model**

In [28]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [29]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [30]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [31]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.6009    0.7341    0.6608       361
           2     0.7797    0.5823    0.6667       553
           3     0.5847    0.6993    0.6369       306

    accuracy                         0.6566      1220
   macro avg     0.6551    0.6719    0.6548      1220
weighted avg     0.6779    0.6566    0.6575      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3172    0.4586    0.3750       157
           2     0.4437    0.2827    0.3454       237
           3     0.2534    0.2846    0.2681       130

    accuracy                         0.3359       524
   macro avg     0.3381    0.3420    0.3295       524
weighted avg     0.3586    0.3359    0.3351       524



In [32]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.515574,0.515574,0.568807,0.511937,0.349237,0.349237,0.365260,0.341332
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.425573,0.425573,0.332235,0.320570
2,SVM,0.656557,0.656557,0.677867,0.657480,0.335878,0.335878,0.358591,0.335077


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5067    0.5208    0.5137       361
           2     0.5775    0.7414    0.6492       553
           3     0.5108    0.2320    0.3191       306

    accuracy                         0.5484      1220
   macro avg     0.5317    0.4981    0.4940      1220
weighted avg     0.5398    0.5484    0.5263      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2455    0.2611    0.2531       157
           2     0.4507    0.5401    0.4914       237
           3     0.1918    0.1077    0.1379       130

    accuracy                         0.3492       524
   macro avg     0.2960    0.3030    0.2941       524
weighted avg     0.3250    0.3492    0.3323       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.526230,0.526230,0.651813,0.442314,0.427481,0.427481,0.338740,0.321962
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.429389,0.429389,0.328686,0.329094
2,Adaboost,0.596721,0.596721,0.599029,0.590531,0.362595,0.362595,0.328523,0.336370
3,GBM,0.984426,0.984426,0.984944,0.984438,0.391221,0.391221,0.348875,0.355065
4,SVM,0.681967,0.681967,0.681725,0.678142,0.375954,0.375954,0.364371,0.369055
5,KNN,0.548361,0.548361,0.539814,0.526320,0.349237,0.349237,0.324987,0.332287


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3687    0.7036    0.4838       361
           2     0.6597    0.4557    0.5390       553
           3     0.5570    0.2712    0.3648       306

    accuracy                         0.4828      1220
   macro avg     0.5285    0.4768    0.4626      1220
weighted avg     0.5478    0.4828    0.4790      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2881    0.5414    0.3761       157
           2     0.4936    0.3249    0.3919       237
           3     0.2740    0.1538    0.1970       130

    accuracy                         0.3473       524
   macro avg     0.3519    0.3400    0.3217       524
weighted avg     0.3775    0.3473    0.3388       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.526230,0.526230,0.651813,0.442314,0.427481,0.427481,0.338740,0.321962
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.429389,0.429389,0.328686,0.329094
2,Adaboost,0.596721,0.596721,0.599029,0.590531,0.362595,0.362595,0.328523,0.336370
3,GBM,0.984426,0.984426,0.984944,0.984438,0.391221,0.391221,0.348875,0.355065
4,SVM,0.681967,0.681967,0.681725,0.678142,0.375954,0.375954,0.364371,0.369055
5,KNN,0.548361,0.548361,0.539814,0.526320,0.349237,0.349237,0.324987,0.332287
6,GNB,0.482787,0.482787,0.547824,0.479002,0.347328,0.347328,0.377547,0.338807
